# Lab 3: Data Cleaning and Visualization

## Course Policies

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** at the top
of your solution.

## Due Date

** This assignment is due Tuesday 6th Feb at 1:00AM **


In this lab, you will be working on the Berkeley Police Department - Calls Data https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5

To avoid the issues met in `hw2.ipynb`, you will not be working directly on data from this URL, but on a snapshot of the data, to make sure that we all work with the same data files.

The data contains some interesting information but you may need to clean it a bit first.

You may find the material in the lecture helpful.

## Setup

Note that after activating matplotlib to display figures inline via the IPython magic `%matplotlib inline`, we configure a custom default figure size. Virtually every default aspect of matplotlib [can be customized](https://matplotlib.org/users/customizing.html).

In [ ]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

## Question 0
### Question 0a
We will reuse the `fetch_and_cache` utility.
Import it from your `utils.py` (which should be in the `lab02` folder) or redefine it below.

In [ ]:
# Import or redefine the function below

### BEGIN SOLUTION
import requests
from pathlib import Path

def fetch_and_cache(data_url, file, data_dir="data", force=False):
    """
    Download and cache a url and return the file object.
    
    data_url: the web address to download
    file: the file in which to save the results.
    data_dir: (default="data") the location to save the data
    force: if true the file is always re-downloaded 
    
    return: The pathlib.Path object representing the file.
    """
    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok = True)
    file_path = data_dir / Path(file)
    # If the file already exists and we want to force a download then
    # delete the file first so that the creation date is correct.
    if force and file_path.exists():
        file_path.unlink()
    if force or not file_path.exists():
        print('Downloading...', end=' ')
        resp = requests.get(data_url)
        with file_path.open('wb') as f:
            f.write(resp.content)
        print('Done!')
    else:
        import time 
        last_modified_time = time.ctime(file_path.stat().st_mtime)
        print("Using cached version last modified (UTC):", last_modified_time)
    return file_path
### END SOLUTION

In [ ]:
# This checks whether you imported/defined fetch_and_cache
import types
assert type(fetch_and_cache) == types.FunctionType

In [ ]:
data_url = 'http://www.ds100.org/sp18/assets/datasets/lab03_data.zip'
file_name = 'lab03_data.zip'

dest_path = fetch_and_cache(data_url=data_url, file=file_name)
print(f'Located at {dest_path}')

We will now directly unzip the ZIP archive and start working with the uncompressed files.

There is no single right answer to whether to work with compressed files in their compressed state or whether to uncompress them on disk permanently. If you e.g. need to work with multiple tools on the same files, or write many notebooks to analyze them, and they are not too large, it may be more convenient to uncompress them once.  But you may also have situations where you find it preferable to work with the compressed data directly.  
Python gives you tools for both approaches, and you should know how to perform both tasks in order to choose the one that best suits the problem at hand.

---

Here, the ZIP archive contains a `data` folder with a few files in it. Contrary to `hw2.ipynb`, it is the archive of a **folder**, not of a file.

Therefore, we will not specify a destination folder to `extractall`, but directly extract the `data` folder in the zip archive in the current working directory, denoted by `.` in the cell below.

In [ ]:
my_zip = zipfile.ZipFile(dest_path, 'r')
# Note the '.' argument
my_zip.extractall('.')

### Question 0b
Let us use a pathlib functionality to list all files in the `data` directory. You may find useful information in pathlib [docs](https://docs.python.org/3/library/pathlib.html).


Use pathlib's `glob` method to store in the variable `file_names` the list of all files' names in the `data_dir` directory.

Use pathlib's `glob` method to store in the variable `python_files` the sublist of all python files' names in the `data_dir` directory.

In [ ]:
data_dir = Path('data')
file_names = ...
python_files = ...

### BEGIN SOLUTION
# List all files in directory
file_names = [x.name for x in data_dir.glob('*') if x.is_file()]
python_files = [x.name for x in data_dir.glob('*.py') if x.is_file()]
### END SOLUTION

print(file_names)
print(python_files)

In [ ]:
assert isinstance(file_names, list)
assert all([isinstance(file, str) for file in file_names])

### BEGIN HIDDEN TESTS
assert set(file_names) >= {'do_not_readme.md', 'lab03_data.zip', 'dummy.txt', 'go_bears.py', 'hello_world.py', 'Berkeley_PD_-_Calls_for_Service.csv'}
assert set(python_files) >= {'go_bears.py', 'hello_world.py'}
### END HIDDEN TESTS

Let us load in a `pandas.DataFrame` objet the CSV file we have.

In [ ]:
calls = pd.read_csv("data/Berkeley_PD_-_Calls_for_Service.csv")
calls.head()

First let's check basic info. The following methods are very useful.

In [ ]:
calls.info()
calls.describe()

Then we can go over the fields to see what are their meanings. We can first check if all events happened in Berkeley.

In [ ]:
calls.groupby(["City","State"]).count()

### Question 1
It seems OFFENSE and CVLEGEND are both about the type of event. What is the difference between the two? Try to use the `groupby` method to group the two keys.

You should be able to see that CVLEGEND is a broader category of crime. For example, there are three sub-categories in OFFENSE that correspond to LARCENY in CVLEGEND. Find them and save the results in `answer1` (should be a list of strings, you may manually type in the answer).

In [ ]:
# You may use this cell for your scratch work as long as you enter
# in your final answers in the answer1 variable.
answer1 = ...


### BEGIN SOLUTION
answer1 = ["THEFT FELONY (OVER $950)","THEFT FROM PERSON","THEFT MISD. (UNDER $950)"]
calls.groupby(["CVLEGEND", "OFFENSE"]).count()
### END SOLUTION

In [ ]:
assert isinstance(answer1, list)
assert all([isinstance(elt, str) for elt in answer1])
assert len(answer1) == 3
# This makes sure all the values you gave are indeed values of calls['OFFENSE'] (no typo)
assert all([elt in calls['OFFENSE'].values for elt in answer1])


### BEGIN HIDDEN TESTS
# Helper for nbgrader tests
def ascii_sum(ans):
    return sum([sum(map(ord, s.strip())) for s in ans])

#assert ascii_sum(answer1) == 4228  # too stringent
assert set([a.strip().upper() for a in answer1]) == set(['THEFT FELONY (OVER $950)', 'THEFT FROM PERSON', 'THEFT MISD. (UNDER $950)'])
### END HIDDEN TESTS

### Question 2

#### Question 2a

What are the five crime types of CVLEGEND that have the most crime events? You may need to use `value_counts` to find the answer.
Save your results into `answer2a` as a list of strings

In [ ]:
answer2a = ...

### BEGIN SOLUTION
answer2a = list(calls["CVLEGEND"].value_counts().head(5).keys())
### END SOLUTION

In [ ]:
assert isinstance(answer2a, list)
assert all([isinstance(elt, str) for elt in answer2a])
# This makes sure all the values you gave are indeed values of the DataFrame (no typo)
assert all([elt in calls['CVLEGEND'].values for elt in answer2a])

### BEGIN HIDDEN TESTS
assert set(answer2a) == set(['LARCENY', 'BURGLARY - VEHICLE', 'VANDALISM', 'DISORDERLY CONDUCT', 'ASSAULT'])
### END HIDDEN TESTS

### Pandas Plotting

Pandas offers basic functionality for plotting.  Take a look at the [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html).

In this lab we will use a few of the basic functions.

#### Question 2b

In the following example we use the `barh` plot type to visually display the value counts for `CVLEGEND`.

In [ ]:
calls['CVLEGEND'].value_counts().plot(kind='barh', colormap='cubehelix');

Look at the documentation and try to construct a regular bar plot in which the bars are oriented vertically.

In [ ]:
### BEGIN SOLUTION
calls['CVLEGEND'].value_counts().plot(kind='bar', colormap='cubehelix')
### END SOLUTION


# Leave this for grading purposes
ax_2b = plt.gca()

In [ ]:
import matplotlib 
# Check ordering
bars = [rect.get_height() for rect in ax_2b.patches 
        if isinstance(rect, matplotlib.patches.Rectangle) and rect.get_x() != 0.0
       ]
# Check values
assert set(bars) == set(calls['CVLEGEND'].value_counts().data)

### Question 3a

The CVDOW field isn't named helpfully and it is hard to see the meaning from the data alone. According to the website, CVDOW is actually indicating the day that events happend. 0->Sunday, 1->Monday ... 6->Saturday. 

Add a new column `Day` into the `calls` dataframe that has the string weekday (eg. 'Sunday') for the corresponding value in CVDOW.

**Hint:** *Try evaluating `days[calls["CVDOW"]]`.  Can you assign this to the new column `calls["Day"]`?*

In [ ]:
days = np.array(["Sunday", "Monday", "Tuesday", 
                 "Wednesday", "Thursday", "Friday",
                 "Saturday"])

### BEGIN SOLUTION
calls["Day"] = days[calls["CVDOW"]]
### END SOLUTION

In [ ]:
assert set(calls["Day"]) == {'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'}
assert calls["Day"][0] == 'Sunday'

### BEGIN HIDDEN TESTS
assert list(calls["Day"][:5]) == ['Sunday', 'Thursday', 'Thursday', 'Thursday', 'Tuesday']
### END HIDDEN TESTS

### Question 3b


Construct a vertical bar plot with the count of the number of entries in the table for each day of the week **ordered by the day of the week** (eg. `Sunday`, `Monday`, ...). 

**Hint:** *Try evaluating `value_counts()[days]` to order the days.*


In [ ]:
### BEGIN SOLUTION
calls['Day'].value_counts()[days].plot(kind='bar', colormap='cubehelix')
### END SOLUTION

# Leave this for grading purposes
ax_3b = plt.gca()

In [ ]:
import matplotlib 
# Check ordering
assert np.alltrue(np.array([l.get_text() for l in ax_3b.xaxis.get_ticklabels()]) == days)
bars = [rect.get_height() for rect in ax_3b.get_children() 
        if isinstance(rect, matplotlib.patches.Rectangle) and rect.get_x() != 0.0
       ]
# Check values
assert np.alltrue(np.array(bars) == calls['Day'].value_counts()[days].data)

It seems Saturday and Friday have slightly more crimes than the others, while Sunday is the lowest. But the difference does not look significant.  
We can break down into some particular types of events to see their distribution. For example, please make a `countplot` for the "NOISE VIOLATION". Which two days are the peak for "NOISE VIOLATION"?  
Put your results into `answer3b` as a list of strings, manually if you prefer.

In [ ]:
# TODO: make some plots here
answer3b = ...

### BEGIN SOLUTION
answer3b = ["Friday", "Saturday"]
calls[calls["CVLEGEND"] == "NOISE VIOLATION"]['Day'].value_counts()[days].plot(kind='bar', colormap='cubehelix');
### END SOLUTION

In [ ]:
assert isinstance(answer3b, list)
assert all([isinstance(elt, str) for elt in answer3b])
assert all([elt in calls['Day'].values for elt in answer3b])

### BEGIN HIDDEN TESTS
assert set([a.lower() for a in answer3b]) == {"friday", "saturday"}
### END HIDDEN TESTS

### Question 4a
Now let's look at the EVENTTM column which indicates the time for events. Since it contains hour and minute information, let's extract the hour info and create a new column named `Hour` in the `calls` dataframe. You should save the hour as an `int`. Then plot the frequency of each hour in the table (i.e., `value_counts()`) sorted by the hour of the day (i.e., `sort_index()`).

You will want to look into how to use:

* [Series.str.slice](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.slice.html#pandas.Series.str.slice) to select the substring.
* [Series.astype](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.astype.html) to change the type.

**Hint:** *The `str` helper member of a series can be used to grab substrings.  For example, `calls["EVENTTM"].str.slice(3,5)` returns the minute of each hour of the `EVENTTM`.*

In [ ]:
### BEGIN SOLUTION
calls["Hour"] = calls["EVENTTM"].str.slice(0,2).astype('int')
calls["Hour"].value_counts().sort_index().plot(kind='bar', colormap='cubehelix');
### END SOLUTION

In [ ]:
assert 'Hour' in calls.columns
assert set(calls["Hour"]) == set(range(24))

### BEGIN HIDDEN TESTS
list(calls["Hour"][:5]) == [6, 8, 18, 17, 18]
assert np.allclose(calls['Hour'].mean(), 13.52451)
### END HIDDEN TESTS

### Question 4b

Now let's construct a boxplot which examines the hour of day of each crime broken down by the `CVLEGEND` value.  To construct this plot you will want to read the [DataFrame.boxplot](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.boxplot.html) documentation.

Which crime type appears to have the largest interquartile range? Put your results into `answer4b` as a string.

**Hint:** *To construct the box plot you will need to specify the `column` to summarize (e.g., `"Hour"`) as well as the criterion to group `by` (e.g., `"CVLEGEND"`).*

**Hint:** *To rotate the labels so they are readable you migth want to set `rot` to `90`.*

**Hint:** *To make the plot more readable you might want to set `figsize=(10,8)`.*

In [ ]:
answer4b = ...

### BEGIN SOLUTION
answer4b = "NOISE VIOLATION"
calls.boxplot(column="Hour", by='CVLEGEND', rot=90, figsize=(10,8));
### END SOLUTION

In [ ]:
assert isinstance(answer4b, str)
### BEGIN HIDDEN TESTS
assert answer4b.upper() == "NOISE VIOLATION"
### END HIDDEN TESTS

## Congratulations

Congrats! You are finished with this assignment.